# Exposing services via ingress

1. Use yaml configs instead of cli commands
1. Create demo nginx setup
1. Create new service
1. Patch ingress config to expose new service via load balancer

In [61]:
!k3d create --publish 80:80 --workers 2

2019/06/22 17:58:42 Created cluster network with ID e90e14236dd1c8a278c64d98f5d7504d59936e8bfe5e8a0c782e1afd366261cc
2019/06/22 17:58:42 Creating cluster [k3s-default]
2019/06/22 17:58:42 Creating server using docker.io/rancher/k3s:v0.5.0...
2019/06/22 17:58:43 Booting 2 workers for cluster k3s-default
2019/06/22 17:58:43 Created worker with ID 8a29998e2d86fd1918a0b10355e38a6b6a8827a39bd107c978ae64251751f8ab
2019/06/22 17:58:44 Created worker with ID 8a01066b7a5c41dc3e01b2fb215bc6d305f7b7ed95d6d329537b2d3ef6deb1db
2019/06/22 17:58:44 SUCCESS: created cluster [k3s-default]
2019/06/22 17:58:44 You can now use the cluster with:

export KUBECONFIG="$(k3d get-kubeconfig --name='k3s-default')"
kubectl cluster-info


In [63]:
!cp "$(k3d get-kubeconfig --name='k3s-default')" ~/.kube/config

Test ingress like this example
 https://github.com/rancher/k3d/issues/11#issuecomment-492822086

In [64]:
%%bash
cat <<EOF | kubectl create -f -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: nginx-demo-dom
  labels:
    app: nginx-demo-dom
spec:
  replicas: 2
  selector:
    matchLabels:
      app: nginx-demo-dom
  template:
    metadata:
      labels:
        app: nginx-demo-dom
    spec:
      containers:
      - name: nginx-demo-dom
        image: nginx:latest
        ports:
        - containerPort: 80
---
apiVersion: v1
kind: Service
metadata:
  name: nginx-demo-dom
  labels:
    app: nginx-demo-dom
spec:
  ports:
    - port: 8081
      targetPort: 80
      name: http
  selector:
    app: nginx-demo-dom
---
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: nginx-demo-dom
  annotations:
    ingress.kubernetes.io/ssl-redirect: "false"
spec:
  rules:
  - host: k3d-ingress-demo.com
    http:
      paths:
      - backend:
          serviceName: nginx-demo-dom
          servicePort: 8081
EOF

deployment.apps/nginx-demo-dom created
service/nginx-demo-dom created
ingress.extensions/nginx-demo-dom created


query deployments unttill available becomes "2"

In [67]:
!kubectl get deployments

NAME             READY   UP-TO-DATE   AVAILABLE   AGE
nginx-demo-dom   2/2     2            2           36s


In [68]:
!kubectl get svc

NAME             TYPE        CLUSTER-IP    EXTERNAL-IP   PORT(S)    AGE
kubernetes       ClusterIP   10.43.0.1     <none>        443/TCP    2m12s
nginx-demo-dom   ClusterIP   10.43.47.13   <none>        8081/TCP   39s


In [73]:
## make sure it works by sending request to our localhost:80 and specifying which host we are targeting. 

In [74]:
!curl -H "Host: k3d-ingress-demo.com" http://localhost

<!DOCTYPE html>
<html>
<head>
<title>Welcome to nginx!</title>
<style>
    body {
        width: 35em;
        margin: 0 auto;
        font-family: Tahoma, Verdana, Arial, sans-serif;
    }
</style>
</head>
<body>
<h1>Welcome to nginx!</h1>
<p>If you see this page, the nginx web server is successfully installed and
working. Further configuration is required.</p>

<p>For online documentation and support please refer to
<a href="http://nginx.org/">nginx.org</a>.<br/>
Commercial support is available at
<a href="http://nginx.com/">nginx.com</a>.</p>

<p><em>Thank you for using nginx.</em></p>
</body>
</html>


## create a new service based on the `whoami`container

In [75]:
%%bash
cat <<EOF | kubectl create -f  -
apiVersion: apps/v1
kind: Deployment
metadata:
  name: whoami
  labels:
    app: whoami
spec:
  replicas: 4
  selector:
    matchLabels:
      app: whoami
  template:
    metadata:
      labels:
        app: whoami
    spec:
      containers:
      - name: whoami
        image: containous/whoami:latest
        ports:
        - containerPort: 80
---
apiVersion: v1
kind: Service
metadata:
  name: whoami
  labels:
    app: whoami
spec:
  ports:
    - port: 8081
      targetPort: 80
      name: http
  selector:
    app: whoami
EOF

deployment.apps/whoami created
service/whoami created


In [77]:
!kubectl get deployments

NAME             READY   UP-TO-DATE   AVAILABLE   AGE
nginx-demo-dom   2/2     2            2           3m45s
whoami           4/4     4            4           7s


In [78]:
## now we need to update ingress config:

In [79]:
%%bash
cat <<EOF | kubectl replace -f -
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  name: nginx-demo-dom
  annotations:
    ingress.kubernetes.io/ssl-redirect: "false"
spec:
  rules:
  - host: k3d-ingress-demo.com
    http:
      paths:
      - backend:
          serviceName: nginx-demo-dom
          servicePort: 8081
  - host: whoami-demo.com
    http:
      paths:
      - backend:
          serviceName: whoami
          servicePort: 8081 
EOF

ingress.extensions/nginx-demo-dom replaced


In [80]:
!kubectl describe ingress nginx-demo-dom

Name:             nginx-demo-dom
Namespace:        default
Address:          
Default backend:  default-http-backend:80 (<none>)
Rules:
  Host                  Path  Backends
  ----                  ----  --------
  k3d-ingress-demo.com  
                           nginx-demo-dom:8081 (10.42.0.4:80,10.42.2.4:80)
  whoami-demo.com       
                           whoami:8081 (10.42.0.5:80,10.42.1.4:80,10.42.2.5:80 + 1 more...)
Annotations:
  ingress.kubernetes.io/ssl-redirect:  false
Events:                                <none>


Let's do some requests. There are 4 pods and nginx does round-robing balancing

In [86]:
!curl -H "Host: whoami-demo.com" http://localhost

Hostname: whoami-6d996cf96f-88fs2
IP: 127.0.0.1
IP: ::1
IP: 10.42.1.4
IP: fe80::649e:f8ff:fe24:6c57














In [87]:
!curl -H "Host: whoami-demo.com" http://localhost

Hostname: whoami-6d996cf96f-vcxhx
IP: 127.0.0.1
IP: ::1
IP: 10.42.0.5
IP: fe80::d8e2:62ff:feab:5c07














In [88]:
!curl -H "Host: whoami-demo.com" http://localhost

Hostname: whoami-6d996cf96f-z9l5m
IP: 127.0.0.1
IP: ::1
IP: 10.42.2.6
IP: fe80::341e:63ff:fe38:38d2














In [89]:
!curl -H "Host: whoami-demo.com" http://localhost

Hostname: whoami-6d996cf96f-2zzhg
IP: 127.0.0.1
IP: ::1
IP: 10.42.2.5
IP: fe80::b490:97ff:febc:694d














In [90]:
!kubectl get pods

NAME                              READY   STATUS    RESTARTS   AGE
nginx-demo-dom-8487c9cb58-wllv2   1/1     Running   0          5m29s
nginx-demo-dom-8487c9cb58-wnr6p   1/1     Running   0          5m29s
whoami-6d996cf96f-2zzhg           1/1     Running   0          111s
whoami-6d996cf96f-88fs2           1/1     Running   0          111s
whoami-6d996cf96f-vcxhx           1/1     Running   0          111s
whoami-6d996cf96f-z9l5m           1/1     Running   0          111s


## Graceful update via apply

we used `replace` command before, which is suboptimal in prod systems, because K8s will kill resource and recreate it with new config. Better approach would be to use `apply` or `edit`. I'll use apply here only

first, we have to load yaml config

In [91]:
!kubectl get ingress nginx-demo-dom -o yaml 

apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  annotations:
    ingress.kubernetes.io/ssl-redirect: "false"
  creationTimestamp: "2019-06-22T16:00:21Z"
  generation: 2
  name: nginx-demo-dom
  namespace: default
  resourceVersion: "630"
  selfLink: /apis/extensions/v1beta1/namespaces/default/ingresses/nginx-demo-dom
  uid: d6e3f901-9506-11e9-bb0a-0242ac1c0002
spec:
  rules:
  - host: k3d-ingress-demo.com
    http:
      paths:
      - backend:
          serviceName: nginx-demo-dom
          servicePort: 8081
  - host: whoami-demo.com
    http:
      paths:
      - backend:
          serviceName: whoami
          servicePort: 8081
status:
  loadBalancer: {}


copy paste above config and make required changes. Here I will change domain name

In [93]:
%%bash
cat <<EOF | kubectl apply -f -
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  annotations:
    ingress.kubernetes.io/ssl-redirect: "false"
  creationTimestamp: "2019-06-22T16:00:21Z"
  generation: 2
  name: nginx-demo-dom
  namespace: default
  resourceVersion: "630"
  selfLink: /apis/extensions/v1beta1/namespaces/default/ingresses/nginx-demo-dom
  uid: d6e3f901-9506-11e9-bb0a-0242ac1c0002
spec:
  rules:
  - host: k3d-ingress-demo.com
    http:
      paths:
      - backend:
          serviceName: nginx-demo-dom
          servicePort: 8081
  - host: whoami-demo-2.com
    http:
      paths:
      - backend:
          serviceName: whoami
          servicePort: 8081
status:
  loadBalancer: {}
EOF

ingress.extensions/nginx-demo-dom configured


There is a warning `Warning: kubectl apply should be used on resource created by either kubectl create --save-config or kubectl apply`, which happened because of the way we've created ingress before. So either always use `replace` or `apply` or `create --save-config` in order to use applies.

In [95]:
!curl -H "Host: whoami-demo.com" http://localhost

404 page not found


In [96]:
!curl -H "Host: whoami-demo-2.com" http://localhost

Hostname: whoami-6d996cf96f-88fs2
IP: 127.0.0.1
IP: ::1
IP: 10.42.1.4
IP: fe80::649e:f8ff:fe24:6c57














In [97]:
!kubectl get ingress nginx-demo-dom -o yaml

apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  annotations:
    ingress.kubernetes.io/ssl-redirect: "false"
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"extensions/v1beta1","kind":"Ingress","metadata":{"annotations":{"ingress.kubernetes.io/ssl-redirect":"false"},"creationTimestamp":"2019-06-22T16:00:21Z","generation":2,"name":"nginx-demo-dom","namespace":"default","resourceVersion":"630","selfLink":"/apis/extensions/v1beta1/namespaces/default/ingresses/nginx-demo-dom","uid":"d6e3f901-9506-11e9-bb0a-0242ac1c0002"},"spec":{"rules":[{"host":"k3d-ingress-demo.com","http":{"paths":[{"backend":{"serviceName":"nginx-demo-dom","servicePort":8081}}]}},{"host":"whoami-demo-2.com","http":{"paths":[{"backend":{"serviceName":"whoami","servicePort":8081}}]}}]},"status":{"loadBalancer":{}}}
  creationTimestamp: "2019-06-22T16:00:21Z"
  generation: 3
  name: nginx-demo-dom
  namespace: default
  resourceVersion: "771"
  selfLink: /apis/extensions/v1beta1/names

In [98]:
%%bash
cat <<EOF | kubectl apply -f -
apiVersion: extensions/v1beta1
kind: Ingress
metadata:
  annotations:
    ingress.kubernetes.io/ssl-redirect: "false"
    kubectl.kubernetes.io/last-applied-configuration: |
      {"apiVersion":"extensions/v1beta1","kind":"Ingress","metadata":{"annotations":{"ingress.kubernetes.io/ssl-redirect":"false"},"creationTimestamp":"2019-06-22T16:00:21Z","generation":2,"name":"nginx-demo-dom","namespace":"default","resourceVersion":"630","selfLink":"/apis/extensions/v1beta1/namespaces/default/ingresses/nginx-demo-dom","uid":"d6e3f901-9506-11e9-bb0a-0242ac1c0002"},"spec":{"rules":[{"host":"k3d-ingress-demo.com","http":{"paths":[{"backend":{"serviceName":"nginx-demo-dom","servicePort":8081}}]}},{"host":"whoami-demo-2.com","http":{"paths":[{"backend":{"serviceName":"whoami","servicePort":8081}}]}}]},"status":{"loadBalancer":{}}}
  creationTimestamp: "2019-06-22T16:00:21Z"
  generation: 3
  name: nginx-demo-dom
  namespace: default
  resourceVersion: "771"
  selfLink: /apis/extensions/v1beta1/namespaces/default/ingresses/nginx-demo-dom
  uid: d6e3f901-9506-11e9-bb0a-0242ac1c0002
spec:
  rules:
  - host: k3d-ingress-demo.com
    http:
      paths:
      - backend:
          serviceName: nginx-demo-dom
          servicePort: 8081
  - host: whoami-demo.com
    http:
      paths:
      - backend:
          serviceName: whoami
          servicePort: 8081
status:
  loadBalancer: {}
EOF

ingress.extensions/nginx-demo-dom configured


interesting, that for the second time we don't have this warning anymore

In [99]:
# delete cluster
!k3d d 

2019/06/22 18:17:48 Removing cluster [k3s-default]
2019/06/22 18:17:48 ...Removing 2 workers
2019/06/22 18:17:50 ...Removing server
2019/06/22 18:17:52 SUCCESS: removed cluster [k3s-default]
